In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pyodbc
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import dateutil

In [2]:
def database_connection():
    conn = pyodbc.connect(
        r'DRIVER={SQL Server Native Client 11.0};'
        r'SERVER=localhost;'
        r'DATABASE=seis603_pricing_project;'
        r'UID=python_dev;'
        r'PWD=python_dev;'
        )
    cursor = conn.cursor()
    return cursor

db_connection = database_connection()

In [3]:
user_id=100
#items tracking
sql = "EXEC [dbo].[usp_GetCurrentTracking] {}".format(user_id)
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result
#df = pd.Series(list_items)
labels = ['item id','item name']
df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list
print(df)    

   item id             item name
0      101     Adrenaline GTS 17
1      102  Radian R100 Car Seat
2      103               Samsung


In [4]:
#set up user and item we are looking up
item_id = 102


In [5]:
#price history of items
sql = "EXEC [dbo].[usp_GetUserItemPriceHistory] {},{}".format(user_id,item_id)
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result

#putting price history into a data frame
labels = ['item id','item name','Brand','Store ID', 'Store Name','Item Price','Price Date']
price_history_df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list

#pivot data and replace all None values with 0
pvt_price_history =  price_history_df.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)

#unpvit the price 
upvt_price_history = pvt_price_history.unstack().reset_index(name='Item Price')

#replace 0 with None so we can front fill later
upvt_price_history['Item Price'].replace(
    to_replace=[0],
    value=None,
    inplace=True
)

#front fill all the data
upvt_price_history.groupby('Price Date')['Item Price'].fillna(method='bfill') 


# repivot for graphing
pvt_price_history2 =  upvt_price_history.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)



In [6]:
#grab all data from report above
calc_pd = upvt_price_history#[['Price Date','Item Price']]
item_name = price_history_df["item name"].unique()[0]

#Convert slices to be correct data types
calc_pd['Item Price'] = calc_pd['Item Price'].astype('float')
calc_pd['Price Date'] = pd.to_datetime(calc_pd['Price Date'])

#Drop values with 0 (i.e. missing dates and would srew up average calc)
calc_pd = calc_pd[calc_pd['Item Price'] != 0]
#get dates
#dates = calc_pd['Price Date'].drop_duplicates()#.sort_values(by='Price Date',ascending=True)
#calc average
avg_per_date = calc_pd.groupby('Price Date')['Item Price'].mean()
#calc min
min_per_date = calc_pd.groupby('Price Date')['Item Price'].min()
#alc max
max_per_date = calc_pd.groupby('Price Date')['Item Price'].max()



In [7]:
#max daily price along all stores
max_trace = go.Scatter(
    x = list(max_per_date.index),#dates
    y = list(max_per_date),
    name = 'Daily Max Price',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)
#min daily price along all stores
min_trace = go.Scatter(
    x = list(min_per_date.index),#dates
    y = list(min_per_date),
    name = 'Daily Min Price',
    line = dict(
        color = ('rgb(49, 86, 219)'),
        width = 4)
)
#average daily price along all store
average_trace = go.Scatter(
    x = list(avg_per_date.index),#dates
    y = list(avg_per_date),
    name = 'Daily Average Price',
    line = dict(
        color = ('rgb(255, 153, 0)'),
        width = 4)
)

#all the data i.e. the lines in the graph
data = [max_trace, average_trace, min_trace]

#Creating layout of report
layout = go.Layout(
    title='Daily Price Agg between All Atores for {}'.format(item_name),
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Item Price',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-line')